#STEP 1: Parse .conllu

In [1]:
def parse_conllu(filepath):
    sentences = []
    words = []
    tags = []

    with open(filepath, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()

            # Sentence boundary
            if line == "":
                if words:
                    sentences.append((words, tags))
                    words = []
                    tags = []
                continue

            # Ignore comments
            if line.startswith("#"):
                continue

            cols = line.split("\t")

            # Skip multi-word tokens like 1-2
            if "-" in cols[0] or "." in cols[0]:
                continue

            word = cols[1]
            tag = cols[3]

            words.append(word)
            tags.append(tag)

    # Catch last sentence if file doesn't end with newline
    if words:
        sentences.append((words, tags))

    return sentences


In [2]:
train_data = parse_conllu("en_ewt-ud-train.conllu")

print(len(train_data))              # number of sentences
print(train_data[0][0])             # first sentence words
print(train_data[0][1])             # first sentence tags


12544
['Al', '-', 'Zaman', ':', 'American', 'forces', 'killed', 'Shaikh', 'Abdullah', 'al', '-', 'Ani', ',', 'the', 'preacher', 'at', 'the', 'mosque', 'in', 'the', 'town', 'of', 'Qaim', ',', 'near', 'the', 'Syrian', 'border', '.']
['PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'ADJ', 'NOUN', 'VERB', 'PROPN', 'PROPN', 'PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'PUNCT', 'ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT']


#STEP 2: Build Vocabulary & Handle <UNK>

STEP 2.1: Count word frequencies

In [3]:
from collections import Counter

def get_word_frequencies(sentences):
    word_freq = Counter()

    for words, _ in sentences:
        for word in words:
            word_freq[word] += 1

    return word_freq


In [4]:
word_freq = get_word_frequencies(train_data)
print(word_freq.most_common(10))


[('.', 8640), ('the', 8151), (',', 7021), ('to', 5076), ('and', 4855), ('a', 3609), ('of', 3589), ('I', 3123), ('in', 2911), ('is', 2152)]


STEP 2.2: Decide what is “rare”

STEP 2.3: Replace rare words with <UNK>

In [5]:
def replace_rare_words(sentences, word_freq, threshold=1):
    new_sentences = []

    for words, tags in sentences:
        new_words = []
        for word in words:
            if word_freq[word] <= threshold:
                new_words.append("<UNK>")
            else:
                new_words.append(word)
        new_sentences.append((new_words, tags))

    return new_sentences


In [6]:
train_data_unk = replace_rare_words(train_data, word_freq)


In [7]:
print(train_data[0][0])
print(train_data_unk[0][0])


['Al', '-', 'Zaman', ':', 'American', 'forces', 'killed', 'Shaikh', 'Abdullah', 'al', '-', 'Ani', ',', 'the', 'preacher', 'at', 'the', 'mosque', 'in', 'the', 'town', 'of', 'Qaim', ',', 'near', 'the', 'Syrian', 'border', '.']
['Al', '-', 'Zaman', ':', 'American', 'forces', 'killed', 'Shaikh', '<UNK>', 'al', '-', '<UNK>', ',', 'the', 'preacher', 'at', 'the', 'mosque', 'in', 'the', 'town', 'of', 'Qaim', ',', 'near', 'the', 'Syrian', 'border', '.']


STEP 2.4: Build final vocabulary

In [8]:
def build_vocabulary(sentences):
    vocab = set()

    for words, _ in sentences:
        for word in words:
            vocab.add(word)

    return vocab


In [9]:
vocab = build_vocabulary(train_data_unk)

print(len(vocab))
print("<UNK>" in vocab)


9874
True


#step-3

STEP 3.1: Count Initial Tags

In [10]:
from collections import defaultdict

def compute_initial_counts(sentences):
    init_counts = defaultdict(int)
    total_sentences = len(sentences)

    for _, tags in sentences:
        init_counts[tags[0]] += 1

    return init_counts, total_sentences


In [11]:
def compute_initial_probs(init_counts, total_sentences):
    init_probs = {}

    for tag, count in init_counts.items():
        init_probs[tag] = count / total_sentences

    return init_probs


STEP 3.2: Count Tag Frequencies

In [12]:
def compute_tag_counts(sentences):
    tag_counts = defaultdict(int)

    for _, tags in sentences:
        for tag in tags:
            tag_counts[tag] += 1

    return tag_counts


STEP 3.3: Count Transition Frequencies

In [13]:
def compute_transition_counts(sentences):
    trans_counts = defaultdict(lambda: defaultdict(int))

    for _, tags in sentences:
        for i in range(len(tags) - 1):
            prev_tag = tags[i]
            next_tag = tags[i + 1]
            trans_counts[prev_tag][next_tag] += 1

    return trans_counts


STEP 3.4: Count Emission Frequencies

In [14]:
def compute_emission_counts(sentences):
    emit_counts = defaultdict(lambda: defaultdict(int))

    for words, tags in sentences:
        for word, tag in zip(words, tags):
            emit_counts[tag][word] += 1

    return emit_counts


STEP 3.5: Convert Counts → Probabilities

In [15]:
def compute_transition_probs(trans_counts, tag_counts):
    trans_probs = defaultdict(dict)

    for prev_tag in trans_counts:
        for next_tag in trans_counts[prev_tag]:
            trans_probs[prev_tag][next_tag] = (
                trans_counts[prev_tag][next_tag] / tag_counts[prev_tag]
            )

    return trans_probs


In [16]:
def compute_emission_probs(emit_counts, tag_counts):
    emit_probs = defaultdict(dict)

    for tag in emit_counts:
        for word in emit_counts[tag]:
            emit_probs[tag][word] = (
                emit_counts[tag][word] / tag_counts[tag]
            )

    return emit_probs


STEP 3.6: Putting it all together

In [17]:
# Initial
init_counts, total_sentences = compute_initial_counts(train_data_unk)
init_probs = compute_initial_probs(init_counts, total_sentences)

# Tag counts
tag_counts = compute_tag_counts(train_data_unk)

# Transitions
trans_counts = compute_transition_counts(train_data_unk)
trans_probs = compute_transition_probs(trans_counts, tag_counts)

# Emissions
emit_counts = compute_emission_counts(train_data_unk)
emit_probs = compute_emission_probs(emit_counts, tag_counts)


In [18]:
#sanity check
print(init_probs)
print(trans_probs["PRON"])
print(emit_probs["NOUN"])


{'PROPN': 0.12771045918367346, 'PUNCT': 0.035235969387755105, 'NUM': 0.039142219387755105, 'DET': 0.10044642857142858, 'PRON': 0.28212691326530615, 'SCONJ': 0.035634566326530615, 'ADP': 0.04320790816326531, 'NOUN': 0.06194196428571429, 'CCONJ': 0.02311862244897959, 'ADV': 0.07653061224489796, 'ADJ': 0.04097576530612245, 'VERB': 0.06050701530612245, 'AUX': 0.028858418367346938, 'PART': 0.00470344387755102, 'INTJ': 0.03228635204081633, 'SYM': 0.0074936224489795915, 'X': 7.971938775510203e-05}
{'NOUN': 0.13385447341650158, 'AUX': 0.32462386892969963, 'VERB': 0.2645499812603737, 'ADV': 0.053702414734700436, 'ADP': 0.03855008834395245, 'ADJ': 0.03592654066498902, 'PROPN': 0.007067516196391284, 'CCONJ': 0.011297317556352734, 'PRON': 0.029019649836697543, 'SCONJ': 0.006853349038924881, 'PUNCT': 0.05605825346683086, 'DET': 0.019007335225143224, 'PART': 0.012207527975584944, 'NUM': 0.003480216308829041, 'SYM': 0.0010708357873320125, 'INTJ': 0.001606253680998019, 'X': 0.00042833431493280504}
{'f

#STEP 4: Implement Viterbi Decoding

STEP 4.1: Prepare tag list

In [19]:
tags = list(tag_counts.keys())


STEP 4.2: Handle unknown words at test time

In [20]:
def preprocess_sentence(words, vocab):
    return [word if word in vocab else "<UNK>" for word in words]


STEP 4.3: Viterbi decoding for ONE sentence

In [21]:
def viterbi_decode(words, tags, init_probs, trans_probs, emit_probs, vocab):
    words = preprocess_sentence(words, vocab)
    T = len(words)
    N = len(tags)

    # Initialize V and B
    V = [dict() for _ in range(T)]
    B = [dict() for _ in range(T)]

    # ----- Initialization (t = 0) -----
    for tag in tags:
        V[0][tag] = init_probs.get(tag, 0) * emit_probs.get(tag, {}).get(words[0], 0)
        B[0][tag] = None

    # ----- Recursion (t > 0) -----
    for t in range(1, T):
        for curr_tag in tags:
            max_prob = 0
            best_prev_tag = None

            for prev_tag in tags:
                prob = (
                    V[t - 1][prev_tag]
                    * trans_probs.get(prev_tag, {}).get(curr_tag, 0)
                    * emit_probs.get(curr_tag, {}).get(words[t], 0)
                )

                if prob > max_prob:
                    max_prob = prob
                    best_prev_tag = prev_tag

            V[t][curr_tag] = max_prob
            B[t][curr_tag] = best_prev_tag

    # ----- Termination -----
    last_tag = max(V[T - 1], key=V[T - 1].get)

    # ----- Backtracking -----
    best_path = [last_tag]
    for t in range(T - 1, 0, -1):
        best_path.append(B[t][best_path[-1]])

    best_path.reverse()
    return best_path


STEP 4.4: Test Viterbi on ONE sentence

In [22]:
test_sentence_words, test_sentence_tags = train_data_unk[0]

predicted_tags = viterbi_decode(
    test_sentence_words,
    tags,
    init_probs,
    trans_probs,
    emit_probs,
    vocab
)

print("Words:", test_sentence_words)
print("Gold :", test_sentence_tags)
print("Pred :", predicted_tags)


Words: ['Al', '-', 'Zaman', ':', 'American', 'forces', 'killed', 'Shaikh', '<UNK>', 'al', '-', '<UNK>', ',', 'the', 'preacher', 'at', 'the', 'mosque', 'in', 'the', 'town', 'of', 'Qaim', ',', 'near', 'the', 'Syrian', 'border', '.']
Gold : ['PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'ADJ', 'NOUN', 'VERB', 'PROPN', 'PROPN', 'PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'PUNCT', 'ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT']
Pred : ['PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'ADJ', 'NOUN', 'VERB', 'PROPN', 'PROPN', 'PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'PUNCT', 'ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT']


#STEP 5: Decode the Entire Test Set

STEP 5.1: Parse the test data

In [23]:
test_data = parse_conllu("en_ewt-ud-test.conllu")
print(len(test_data))


2077


STEP 5.2: Decode one test sentence (sanity check)

In [24]:
words, gold_tags = test_data[0]

pred_tags = viterbi_decode(
    words,
    tags,
    init_probs,
    trans_probs,
    emit_probs,
    vocab
)

print("Words:", words)
print("Gold :", gold_tags)
print("Pred :", pred_tags)


Words: ['What', 'if', 'Google', 'Morphed', 'Into', 'GoogleOS', '?']
Gold : ['PRON', 'SCONJ', 'PROPN', 'VERB', 'ADP', 'PROPN', 'PUNCT']
Pred : ['PRON', 'SCONJ', 'PROPN', 'PROPN', 'PROPN', 'PROPN', 'PUNCT']


STEP 5.3: Decode ALL test sentences

In [25]:
def decode_test_set(test_data, tags, init_probs, trans_probs, emit_probs, vocab):
    all_gold = []
    all_pred = []

    for words, gold_tags in test_data:
        pred_tags = viterbi_decode(
            words,
            tags,
            init_probs,
            trans_probs,
            emit_probs,
            vocab
        )

        all_gold.extend(gold_tags)
        all_pred.extend(pred_tags)

    return all_gold, all_pred


In [26]:
gold_tags, predicted_tags = decode_test_set(
    test_data,
    tags,
    init_probs,
    trans_probs,
    emit_probs,
    vocab
)


STEP 5.4: Sanity checks

In [27]:
print(len(gold_tags), len(predicted_tags))
print(gold_tags[:20])
print(predicted_tags[:20])


25094 25094
['PRON', 'SCONJ', 'PROPN', 'VERB', 'ADP', 'PROPN', 'PUNCT', 'PRON', 'SCONJ', 'PROPN', 'VERB', 'ADP', 'PRON', 'NOUN', 'PUNCT', 'NOUN', 'PUNCT', 'CCONJ', 'ADV', 'NOUN']
['PRON', 'SCONJ', 'PROPN', 'PROPN', 'PROPN', 'PROPN', 'PUNCT', 'PRON', 'SCONJ', 'PROPN', 'VERB', 'ADP', 'PRON', 'NOUN', 'PUNCT', 'NOUN', 'PUNCT', 'CCONJ', 'ADV', 'NOUN']


#STEP 6: Evaluation

STEP 6.1: Token-level Accuracy

In [28]:
def compute_accuracy(gold_tags, predicted_tags):
    correct = 0
    total = len(gold_tags)

    for g, p in zip(gold_tags, predicted_tags):
        if g == p:
            correct += 1

    return correct / total


In [29]:
accuracy = compute_accuracy(gold_tags, predicted_tags)
print(f"POS Tagging Accuracy: {accuracy:.4f}")


POS Tagging Accuracy: 0.8942


STEP 6.2: Confusion Matrix

In [30]:
from collections import defaultdict

def compute_confusion_matrix(gold_tags, predicted_tags):
    confusion = defaultdict(lambda: defaultdict(int))

    for g, p in zip(gold_tags, predicted_tags):
        confusion[g][p] += 1

    return confusion


In [31]:
confusion = compute_confusion_matrix(gold_tags, predicted_tags)

# Example: what NOUN is confused with
print(confusion["NOUN"])


defaultdict(<class 'int'>, {'NOUN': 3502, 'VERB': 110, 'PROPN': 362, 'NUM': 31, 'ADJ': 95, 'ADV': 6, 'DET': 2, 'AUX': 3, 'SYM': 4, 'X': 4, 'ADP': 4})
